# Capítulo 7 — Intervalos de confianza e inferencia (t-test)

Notas breves centradas en **sintaxis** y **uso del código**.


In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

plt.rcParams["figure.figsize"] = (8, 4.5)
plt.rcParams["figure.dpi"] = 120


## 1) Intervalo de confianza con una muestra (datos locales)

In [ ]:
# Datos: TD87 y TD88 (20 empresas) + variación
TD87 = np.array([10, 1, 6, .45, 1.25, 1.3, 1.06, 3, 8.18, 1.67, .98, 1, .45, 5.03, 8, 9, 18, .28, 7, 3.97])
TD88 = np.array([3, 1, 5, .5, 1.54, 1.5, .8, 2, .67, 1.17, .51, .5, .61, 6.7, 4, 7, 19, .2, 5, 3.83])

variacion = TD88 - TD87
n = len(variacion)

baseTD = pd.DataFrame({
    "Empresa": np.arange(1, n + 1),
    "TD87": TD87,
    "TD88": TD88,
    "Variación": variacion
})
baseTD.head()


In [ ]:
# Promedio muestral
promedio_var = np.mean(variacion)
promedio_var


In [ ]:
# Desv. estándar muestral (ddof=1) y error estándar del promedio
dest_var = np.std(variacion, ddof=1)
se_var = dest_var / np.sqrt(n)

# Cuantil t para IC 95% bilateral
c = stats.t.ppf(0.975, n - 1)

LI95 = promedio_var - c * se_var
LS95 = promedio_var + c * se_var
LI95, LS95


## 2) Prueba t manual (una cola a la izquierda)

In [ ]:
# Estadístico t: (promedio - mu0) / SE, con mu0=0
est_t_manual = promedio_var / se_var

# Valor crítico (cola izquierda, 5%)
valc_1cola_izq = stats.t.ppf(0.05, n - 1)

est_t_manual, valc_1cola_izq


In [ ]:
# Valor p (una cola izquierda): P(T <= t)
valorp_manual = stats.t.cdf(est_t_manual, n - 1)
valorp_manual


## 3) Prueba t automática (ttest_1samp)

In [ ]:
test_auto = stats.ttest_1samp(variacion, popmean=0)  # bilateral
t_auto = test_auto.statistic
p_bilateral = test_auto.pvalue
p_una_cola = p_bilateral / 2

t_auto, p_bilateral, p_una_cola


## 4) Ejemplo práctico (local): variable y en {-1,0,1}

In [ ]:
audit_like = pd.DataFrame({
    "y": [-1, 0, 0, -1, 1, 0, -1, 0, 0, 1,
          0, 0, -1, 0, 1, 0, 0, -1, 0, 0,
          1, 0, -1, 0, 0, 0, -1, 0, 0, 1]
})
audit_like["y"].value_counts()


In [ ]:
# IC 95% y 99% asumiendo normalidad (usa cuantiles de N(0,1))
y = audit_like["y"]

promedio_y = np.mean(y)
n = len(y)
desv_est_y = np.std(y, ddof=1)
se_y = desv_est_y / np.sqrt(n)

c95 = stats.norm.ppf(0.975)
c99 = stats.norm.ppf(0.995)

LI95 = promedio_y - c95 * se_y
LS95 = promedio_y + c95 * se_y

LI99 = promedio_y - c99 * se_y
LS99 = promedio_y + c99 * se_y

(LI95, LS95), (LI99, LS99)


In [ ]:
# Prueba t (dos colas) sobre promedio de y
test_auto = stats.ttest_1samp(y, popmean=0)
t_auto = test_auto.statistic
p_two = test_auto.pvalue

valc_2colas = stats.t.ppf(0.025, n - 1)  # cola izquierda

promedio_y, t_auto, valc_2colas, p_two


## 5) Datos reales (opcional)

En Colab podrías usar datasets educativos (ej. `wooldridge`).  
Aquí se deja comentado para que no falle si no está instalado.


In [ ]:
# !pip -q install wooldridge
# import wooldridge as woo
# from scipy import stats
# import numpy as np
#
# audit = woo.dataWoo("audit")
# y = audit["y"]
# promedio_y = np.mean(y)
# n = len(y)
# se_y = np.std(y, ddof=1) / np.sqrt(n)
# c95 = stats.norm.ppf(0.975)
# print(promedio_y - c95*se_y, promedio_y + c95*se_y)
#
# print(stats.ttest_1samp(y, popmean=0))


## Ejercicios propuestos

1) Con `variacion`, calcula el IC 95% usando t.

**Respuesta esperada:** dos números (LI, LS).


In [ ]:
# Escribe tu solución aquí


2) Con `variacion`, calcula `t = promedio/SE` y el valor crítico (cola izq 5%).

**Respuesta esperada:** dos números negativos.


In [ ]:
# Escribe tu solución aquí


3) Calcula el valor p (cola izq) con `stats.t.cdf(t, n-1)`.

**Respuesta esperada:** un número entre 0 y 1, típicamente pequeño.


In [ ]:
# Escribe tu solución aquí


4) Compara el `t` manual con `ttest_1samp(...).statistic`.

**Respuesta esperada:** casi iguales.


In [ ]:
# Escribe tu solución aquí


5) Con `audit_like["y"]`, calcula el IC 99% (normal).

**Respuesta esperada:** intervalo más ancho que el 95%.


In [ ]:
# Escribe tu solución aquí


6) En `audit_like`, ejecuta `ttest_1samp(y,0)` y reporta el `pvalue`.

**Respuesta esperada:** número entre 0 y 1.


In [ ]:
# Escribe tu solución aquí


## Glosario

- **SE**: error estándar del promedio.
- **ddof=1**: ajuste para desviación estándar muestral.
- **ppf**: cuantil (inversa de CDF).
- **t-test 1 muestra**: prueba sobre el promedio contra `mu0`.
